In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')

# Propositional Classifiers:

In this notebook I try to design classifiers that act on the set of all blocks and output a tiebreaker binary label having access only to the features given by Henry in the focal students dataset. Those features are:

1. Total number of students per block (used for normalization purposes)
2. Number of FRL students per block
3. Number of AALPI students per block
4. Number of FRL and AALPI students per block (i.e. intersection of those)
5. Number of FRL or AALPI students per block (i.e. union)

The column names available are:

1. n
2. nFRL and pctFRL
3. nAALPI and pctAALPI
4. nBoth and pctBoth
5. nFocal and pctFocal

The classifiers will evaluate a logical proposition with those features. For example, an "AND" classifier can be of the form:

$$ \text{AALPI} \geq 0.5 \quad \text{and} \quad \text{FRL} \geq 0.7 $$

This classifier will give an equity tiebreaker to a block if and only if that block has over 50% of its students in the AALPI racial group and over 70% of its students receiving FRL.

Currently we do not have a systematic way to think of these types of propositions. But we can evaluate their performance based on false positive and false negative rates. In the case of two parameters (i.e. two numeric comparisons), it is possible to visualize the precision-recall curve.

## 1. Class Syntax

We load the propositional classifier classes from the modelling library:

In [ ]:
from src.d04_modeling import propositional_classifier as pc

RMK: The first classifier will take some extra seconds to be initialized in order for the data to load

In [ ]:
load_data = pc.andClassifier([], user="gabriel")

In its most general form, the propositional classifier class takes as an input a list of features that we will be using for evaluation, a list of logical operators ("and" or "or"), and a list of comparisors ($\geq$, $\leq$, =). By default, this comparisor lists is a sequence of $\geq$ since that is the most likely case. The lists must be in the order of the statement we want to construct, and notice that there will always be one less operator than features. For example:

In [ ]:
pc1 = pc.PropositionalClassifier(["pctAALPI", "pctFRL", "nBoth"], ["and", "or"], user="gabriel")
pc1.statement

Note that the parameters are not required upon initialization. Rather, the statement is constructed so we can input parameters when doing the predictions. This way we can vary parameters and build precision-recall cruves.

Simple and/or classifiers have their own child class, in which we only need to pass the features (and comparisors if not default):

In [ ]:
pc2 = pc.andClassifier(["pctAALPI", "pctFRL"], user="gabriel")
pc2.statement

In [ ]:
pc3 = pc.orClassifier(["pctAALPI", "pctFRL"], user="gabriel")
pc3.statement

Some logical statements need parentheses. Some do not, but we would rather read them with parentheses as that is easier (for example, the first example pc1 is hard to interpret without parenthesis---computer evaluates it in order). Simply pass a tuple of features as an element. Note that operands must still be of the correct length!

In [ ]:
pc4 = pc.PropositionalClassifier(["pctAALPI", ("pctFRL", "pctBoth"), "nBoth"], ["or", "and", "or"], user="gabriel")
pc4.statement

Once we have initialized our statement we can use the get_solution_set method with the appropriate parameters to do a round of prediction.

In [ ]:
params1 = [0.6, 0.6, 6] #parameters must match the features passed, in the order. Note the scale.
pred1 = pc1.get_solution_set(params1)
pred1

This index object tells us which blocks receive the tiebreaker. We can visualize the result in the San Francisco map:

In [ ]:
ax = pc1.plot_map(params1)

The confusion matrix of this classifier can be retrieved using the get_confusion_matrix method:

In [ ]:
cm1 = pc1.get_confusion_matrix(params1)
cm1

Using that we can retrive any rate for evaluating purposes. We can get the FPR and FNR for example:

In [ ]:
print("False positive rate is {0:.2f} %".format(100*pc1.fpr(params1)))
print("False negative rate is {0:.2f} %".format(100*pc1.fnr(params1)))

To interpret the above map: we wanted the FRL percentage to be very high (above 80%) and the AALPI percentage to be at least half; OR if there were at least 10 students in a block in the intersection count, the block would be given a tiebreaker regardless of its relative composition. We can see in the map that due to the AALPI criterion having to be satisfied most blocks that received a tiebreaker are in the SouthEast (where racial minorities are more concentrated). This criterion is very restrictive: the false negative rate is super high, meaning that we "missed" a lot of focal students. However, very few non-focal students received an advantage (around 15%).

## 2. Exploring the parameter space

Ideally we would like to explore several points for the trade-off between FP and FN.

In one-dimensional parameter spaces (i.e. only one feature is pased to the classifier, so that we have only one parameter) this can be done via analysis of the ROC curve (similar to precision-recall):

In [ ]:
pc5 = pc.PropositionalClassifier(["pctBoth"], [], user="gabriel")

In [ ]:
pc5.statement

In [ ]:
params_arr = [x for x in np.linspace(0, 1, num=100)]
ROC5_df = pc5.get_roc(params_arr)
ROC5_df

In [ ]:
pc5.plot_roc(params_arr)

In two-dimensional parameter spaces (i.e. only two features are pased to the classifier, so that we have only two parameters) this can be done via analysis of two matrices of false positives and false negatives. This would be equivalent to a ROC surface.

In [ ]:
pc2.statement

In [ ]:
params_arr2 = [x/10 for x in range(11)]
pc2.plot_heatmap(params_arr2)

We can fix the parameter for the AALPI rate at $50\%$ and then plot the ROC curve for the FRL parameter:

In [ ]:
params_arr2FRL = [[0.5, x/10] for x in range(11)]
pc2.plot_roc(params_arr2FRL)

An alternative is to fix all but one parameters of the propositions so that we can build a ROC curve. Using:

$$ \left(\text{AALPI } \geq 50\%  \quad \text{and} \quad \text{FRL } \geq 60\%\right)\quad\text{or}\quad \text{BOTH } (\%)\geq \gamma$$

In [ ]:
pc6 = pc.PropositionalClassifier([("pctAALPI", "pctFRL"), "pctBoth"], ["and", "or"], user="gabriel")
pc6.statement

In [ ]:
params_arr6 = [[0.5, 0.6, x] for x in np.linspace(0, 1, num=100)]
pc6.plot_roc(params_arr6)

Note that because we are looking at a subset of the parameter space, we cannot acheive all possible values of FPR/TPR

## 3. Comparing the Performance of Different Propositions:

For the one-parameter case:

In [ ]:
fig, ax = plt.subplots(figsize=(25, 25))

ax.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
param_arr = [x/10 for x in range(11)]

for col in ["pctFocal", "pctBoth", "pctAALPI", "pctFRL"]:
    classifier = pc.andClassifier([col])
    df = classifier.get_roc(param_arr)
    data_fpr = df["fpr"].values
    data_tpr = df["tpr"].values
    ax.plot(data_fpr, data_tpr, linestyle='solid', label=col)

ax.legend(loc="lower right")
plt.legend(fontsize=25)

ax.set_xlabel('False Positive Rate', fontsize=20)
ax.set_ylabel('True Positive Rate', fontsize=20)
ax.set_title("ROC curve for naive classification based on a single feature", fontsize=30)

plt.show()

Using the above approach, let's try to compare the AND and the OR rule for ALLPI and FRL students. We will plot the ROCs for several values of the AALPI parameter.

In [ ]:
c1 = pc.andClassifier(["pctAALPI", "pctFRL"])
c2 = pc.orClassifier(["pctAALPI", "pctFRL"])

In [ ]:
fig, Axes = plt.subplots(figsize=(50,50), nrows=2, ncols=2)

for alpha_aalpi, ax in zip([0, 0.25, 0.5, 0.75], [Axes[0,0], Axes[0, 1], Axes[1,0], Axes[1,1]]):

    ax.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
    
    param_arr = [[alpha_aalpi, x/10] for x in range(11)]
    
    df1 = c1.get_roc(param_arr)
    data_fpr1 = df1["fpr"].values
    data_tpr1 = df1["tpr"].values
    ax.plot(data_fpr1, data_tpr1, linestyle='solid', label="AND Classifier")
    
    df2 = c2.get_roc(param_arr)
    data_fpr2 = df2["fpr"].values
    data_tpr2 = df2["tpr"].values
    ax.plot(data_fpr2, data_tpr2, linestyle='solid', label="OR Classifier")
    
    ax.legend(loc="lower right")
    plt.legend(fontsize=25)

    ax.set_xlabel('False Positive Rate', fontsize=20)
    ax.set_ylabel('True Positive Rate', fontsize=20)
    ax.set_title("ROC curve for pctAALPI > " + str(alpha_aalpi), fontsize=30)

plt.show()